In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('GOOG.csv', date_parser = True)
data.tail()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8968\930745223.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  data = pd.read_csv('GOOG.csv', date_parser = True)


,Date,Open,High,Low,Close,Adj Close,Volume
3804,2019-09-30,1220.969971,1226.000000,1212.300049,1219.000000,1219.000000,1404100
3805,2019-10-01,1219.000000,1231.229980,1203.579956,1205.099976,1205.099976,1273500
3806,2019-10-02,1196.979980,1196.979980,1171.290039,1176.630005,1176.630005,1615100
3807,2019-10-03,1180.000000,1189.060059,1162.430054,1187.829956,1187.829956,1621200
3808,2019-10-04,1191.890015,1211.439941,1189.170044,1209.000000,1209.000000,1021092


In [3]:
data_training = data[data['Date']<'2019-01-01'].copy()
data_test = data[data['Date']>='2019-01-01'].copy()

In [4]:
data_training = data_training.drop(['Date', 'Adj Close'], axis = 1)


In [5]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)
data_training

array([[3.30294890e-04, 9.44785459e-04, 0.00000000e+00, 1.34908021e-04,
        5.43577404e-01],
       [7.42148227e-04, 2.98909923e-03, 1.88269054e-03, 3.39307537e-03,
        2.77885613e-01],
       [4.71386886e-03, 4.78092896e-03, 5.42828241e-03, 3.83867225e-03,
        2.22150736e-01],
       ...,
       [7.92197108e-01, 8.11970141e-01, 7.90196475e-01, 8.15799920e-01,
        2.54672037e-02],
       [8.18777193e-01, 8.21510648e-01, 8.20249255e-01, 8.10219301e-01,
        1.70463908e-02],
       [8.19874096e-01, 8.19172449e-01, 8.12332341e-01, 8.09012935e-01,
        1.79975186e-02]])

In [6]:
data_training[0:10]


array([[3.30294890e-04, 9.44785459e-04, 0.00000000e+00, 1.34908021e-04,
        5.43577404e-01],
       [7.42148227e-04, 2.98909923e-03, 1.88269054e-03, 3.39307537e-03,
        2.77885613e-01],
       [4.71386886e-03, 4.78092896e-03, 5.42828241e-03, 3.83867225e-03,
        2.22150736e-01],
       [4.91367646e-03, 4.01532941e-03, 3.15578542e-03, 1.98678849e-03,
        1.85522018e-01],
       [2.35285614e-03, 2.54928676e-03, 3.28434064e-03, 2.44873974e-03,
        1.11762967e-01],
       [2.34877785e-03, 2.52892558e-03, 3.60779701e-03, 3.22955376e-03,
        8.62763771e-02],
       [3.63326671e-03, 2.80177162e-03, 4.03492722e-03, 2.51005881e-03,
        7.55243925e-02],
       [2.48334262e-03, 1.52712947e-03, 2.50886935e-03, 8.17608079e-04,
        6.31682127e-02],
       [1.26817570e-03, 8.02253103e-04, 2.57107531e-03, 9.64778600e-04,
        5.97732318e-02],
       [1.43128522e-03, 5.00900100e-04, 1.53849690e-03, 9.81131336e-05,
        1.11151095e-01]])

In [7]:
X_train = []
y_train = []

In [8]:
for i in range(60, data_training.shape[0]):
    X_train.append(data_training[i-60:i])
    y_train.append(data_training[i, 0])

In [9]:
X_train, y_train = np.array(X_train), np.array(y_train)


In [10]:
X_train.shape


(3557, 60, 5)

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [12]:
regressior = Sequential()

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressior.add(Dropout(0.2))

regressior.add(LSTM(units = 120, activation = 'relu'))
regressior.add(Dropout(0.2))

regressior.add(Dense(units = 1))

C:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
regressior.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 60, 60)              │          15,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 60, 60)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 60, 60)              │          29,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 60, 60)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 60, 80)              │          45,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 60, 80)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 120)                 │          96,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             121 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 186,601 (728.91 KB)

 Trainable params: 186,601 (728.91 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
regressior.compile(optimizer='adam', loss = 'mean_squared_error')


In [15]:
regressior.fit(X_train, y_train, epochs=50, batch_size=32)


Epoch 1/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - loss: 0.0379
Epoch 2/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - loss: 0.0020
Epoch 3/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 69ms/step - loss: 0.0023
Epoch 4/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - loss: 0.0021
Epoch 5/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 70ms/step - loss: 0.0019
Epoch 6/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 68ms/step - loss: 0.0019
Epoch 7/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - loss: 0.0017
Epoch 8/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - loss: 0.0015
Epoch 9/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - loss: 0.0014
Epoch 10/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - loss: 0.0017
Epoch 11/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 76ms/step - loss: 0.0012
Epoch 12/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - loss: 0.0015
Epoch 13/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 0.0012
Epoch 14/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - loss: 0.0012
Epoch 15/50
112/112 ━━━━━━━━━━━━━━━━━━━━ 

In [16]:
data_test.head()

data_training = data_test.drop(['Date', 'Adj Close'], axis = 1)

data_training=pd.DataFrame(data_test, columns =['Open', 'High', 'Low','Close','Volume'])

In [17]:
#data_training.tail(60)
#data_train

data_training.tail(60)

,Open,High,Low,Close,Volume
3749,1143.989990,1147.339966,1138.780029,1144.900024,864000
3750,1146.859985,1150.819946,1139.400024,1150.339966,903800
3751,1146.000000,1158.579956,1145.000000,1153.579956,1238800
3752,1150.969971,1158.359985,1145.770020,1146.349976,1170000
3753,1141.739990,1147.604980,1132.729980,1146.329956,1291300
3754,1148.189941,1151.140015,1129.619995,1130.099976,1647200
3755,1133.449951,1139.250000,1124.239990,1138.069946,1301500
3756,1144.000000,1146.900024,1131.800049,1146.209961,1093700
3757,1131.900024,1144.000000,1126.989990,1137.810059,1589800
3758,1137.819946,1141.699951,1120.920044,1132.119995,2209800


In [18]:
past_60_days = data_training.tail(60)
past_60_days


,Open,High,Low,Close,Volume
3749,1143.989990,1147.339966,1138.780029,1144.900024,864000
3750,1146.859985,1150.819946,1139.400024,1150.339966,903800
3751,1146.000000,1158.579956,1145.000000,1153.579956,1238800
3752,1150.969971,1158.359985,1145.770020,1146.349976,1170000
3753,1141.739990,1147.604980,1132.729980,1146.329956,1291300
3754,1148.189941,1151.140015,1129.619995,1130.099976,1647200
3755,1133.449951,1139.250000,1124.239990,1138.069946,1301500
3756,1144.000000,1146.900024,1131.800049,1146.209961,1093700
3757,1131.900024,1144.000000,1126.989990,1137.810059,1589800
3758,1137.819946,1141.699951,1120.920044,1132.119995,2209800


In [19]:
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()
import pandas as pd

# Assuming past_60_days and data_test are DataFrame objects
# Concatenate past_60_days and data_test vertically
df = pd.concat([past_60_days, data_test], ignore_index=True)

# Drop columns 'Date' and 'Adj Close'
df = df.drop(['Date', 'Adj Close'], axis=1)

# Display the first few rows of the resulting DataFrame
df.head()


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
#inputs = scaler.transform(df)
#inputs=df.to_numpy()
#inputs



inputs = scaler.transform(df)
inputs

In [ ]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])

In [ ]:
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

In [ ]:
y_pred = regressior.predict(X_test)


In [ ]:
scaler.scale_


In [ ]:
scale = 1/8.18605127e-04
scale

In [ ]:
y_pred = y_pred*scale
y_test = y_test*scale



In [ ]:
# Visualising the results
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'red', label = 'Real Google Stock Price')
plt.plot(y_pred, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()